In [3]:
import os

training_data_path = "/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train"
validation_data_path = "/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Validation"

print(f" Training Subfolders: {os.listdir(training_data_path)}")
print(f" Validation Subfolders: {os.listdir(validation_data_path)}")

 Training Subfolders: ['Female', 'Male']
 Validation Subfolders: ['Female', 'Male']


# Split The Dataset Into Training & Validation

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64

data_generator = ImageDataGenerator(
                                     rescale=1./255,
                                     validation_split=0.2
                                    )

training_data = data_generator.flow_from_directory(
    training_data_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

validation_data = data_generator.flow_from_directory(
    validation_data_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print(training_data.class_indices)

2025-06-21 08:42:19.789168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750495339.964218      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750495340.013899      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
{'Female': 0, 'Male': 1}


# Use MobileNetV2 As Base For Gender Classification

In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

mobilenet_v2 = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
mobilenet_v2.trainable = False

x = mobilenet_v2.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(2, activation="softmax")(x)

gender_classification_model = Model(
    inputs=mobilenet_v2.input,
    outputs=output
)
gender_classification_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
gender_classification_model.summary()

I0000 00:00:1750495580.396632      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Train The Gender Classifier By 25 Epochs

In [8]:
EPOCHS = 5
history = gender_classification_model.fit(
    training_data,
    validation_data=validation_data,
    epochs=EPOCHS
)

Epoch 1/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 977s 391ms/step - accuracy: 0.9058 - loss: 0.2315 - val_accuracy: 0.9205 - val_loss: 0.1976
Epoch 2/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1028s 411ms/step - accuracy: 0.9149 - loss: 0.2097 - val_accuracy: 0.9244 - val_loss: 0.1902
Epoch 3/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 720s 288ms/step - accuracy: 0.9171 - loss: 0.2049 - val_accuracy: 0.9233 - val_loss: 0.1901
Epoch 4/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 714s 285ms/step - accuracy: 0.9196 - loss: 0.2011 - val_accuracy: 0.9265 - val_loss: 0.1843
Epoch 5/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 702s 281ms/step - accuracy: 0.9192 - loss: 0.2009 - val_accuracy: 0.9254 - val_loss: 0.1855


In [9]:
loss, accuracy = gender_classification_model.evaluate(validation_data)
print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

354/354 ━━━━━━━━━━━━━━━━━━━━ 58s 164ms/step - accuracy: 0.9274 - loss: 0.1799
Training Loss: 0.1855381280183792
Training Accuracy: 0.9253916144371033


In [10]:
gender_classification_model.save("/kaggle/working/gender_classifier.h5")

In [11]:
from IPython.display import FileLink

FileLink("gender_classifier.h5") 

/kaggle/working/gender_classifier.h5